In [ ]:
#ffmpeg -i dialog1.wav -f segment -segment_time 30 -c copy parts/out%09d.wav

In [13]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="../Audio-to-speech-aa00874e061f.json"

### Requires
1. Sampling Rate : 16000 
2. Channels : Mono
3. Type : Wav
4. Not more that 1 min (30 second to be safe)

In [40]:
from google.cloud import speech_v1p1beta1 as speech
client = speech.SpeechClient()

speech_file = 'parts/out000000001.wav'

with open(speech_file, 'rb') as audio_file:
    content = audio_file.read()

audio = speech.types.RecognitionAudio(content=content)

config = speech.types.RecognitionConfig(
    encoding=speech.enums.RecognitionConfig.AudioEncoding.LINEAR16,
    sample_rate_hertz=16000,
    language_code='en-US',
    enable_speaker_diarization=True,
    diarization_speaker_count=2)

print('Waiting for operation to complete...')
response = client.recognize(config, audio)

# The transcript within each result is separate and sequential per result.
# However, the words list within an alternative includes all the words
# from all the results thus far. Thus, to get all the words with speaker
# tags, you only have to take the words list from the last result:
result = response.results[-1]

words_info = result.alternatives[0].words

conversation = list()
current = words_info[0].speaker_tag
# Printing out the output:
ongoing = {words_info[0].speaker_tag:[]}
for word_info in words_info:
    #print("word: '{}', speaker_tag: {}".format(word_info.word,word_info.speaker_tag))
    if word_info.speaker_tag != current:
        conversation.append(ongoing)
        ongoing = { word_info.speaker_tag :[] }
        current = word_info.speaker_tag
        ongoing[word_info.speaker_tag].append(word_info.word)
    else:
        ongoing[word_info.speaker_tag].append(word_info.word)
conversation.append(ongoing)
print('Done')

Waiting for operation to complete...
Done


In [41]:
for converse in conversation:
    print( "Speaker {} : {}".format( list(converse.keys())[0], " ".join(converse[list(converse.keys())[0]]) ) )

Speaker 2 : yes I think so that's probably an Wallace wow not long would you live
Speaker 1 : well I'm staying at the YMCA at the moment I'm looking for someone. Do you know of any good places
Speaker 2 : has a spare room in her apartment and she's looking for a flatmate phone number


In [ ]:
import os
import speech_recognition as sr
from tqdm import tqdm

with open("Audio-to-speech-aa00874e061f.json") as f:
    GOOGLE_CLOUD_SPEECH_CREDENTIALS = f.read()
    
r = sr.Recognizer()
files = sorted(os.listdir('parts/'))all_text = []

for f in tqdm(files):
    name = "parts/" + f
    # Load audio file
    with sr.AudioFile(name) as source:
        audio = r.record(source)
    # Transcribe audio file
    text = r.recognize_google_cloud(audio, credentials_json=GOOGLE_CLOUD_SPEECH_CREDENTIALS)
    all_text.append(text)